In [ ]:
!pip install -q peft transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import os

device = 'cuda'
model_name_or_path = "t5-large"
tokenizer_name_or_path = 't5-large'

text_column = 'sentence'
label_column = 'text_label'
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size=8

In [ ]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset['train'].train_test_split(test_size=0.1)
dataset['validation'] = dataset['test']
del dataset['test']

classes = dataset['train'].features['label'].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x['label']]},
    batched=True,
    num_proc=1
)

dataset['train'][0]
{"sentence": "Profit before taxes was EUR 4.0 mn , down from EUR 4.9 mn .", "label": 0, "text_label": "negative"}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'Profit before taxes was EUR 4.0 mn , down from EUR 4.9 mn .',
 'label': 0,
 'text_label': 'negative'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples):
  inputs = examples[text_column]
  targets = examples[label_column]
  model_inputs = tokenizer(inputs, max_length=max_length, padding='max_length',truncation=True,return_tensors='pt')
  labels = tokenizer(targets, max_length=2, padding='max_length', truncation=True, return_tensors='pt')
  labels = labels["input_ids"]
  labels[labels == tokenizer.pad_token_id] = -100
  model_inputs['labels']=labels
  return model_inputs

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset['train'].column_names,
    load_from_cache_file=False,
    desc='Running tokenizer on dataset',
)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets['train']
eval_dataset = processed_datasets['validation']

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size,pin_memory=True)

In [ ]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.1331


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader)*num_epochs),
)

In [ ]:
model = model.to(device)
for epoch in range(num_epochs):
  model.train()
  total_loss = 0
  for step, batch in enumerate(tqdm(train_dataloader)):
    batch = {k: v.to(device) for k,v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    total_loss += loss.detach().float()
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

  model.eval()
  eval_loss = 0
  eval_preds = []
  for step, batch in enumerate(tqdm(eval_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    loss = outputs.loss
    eval_loss += loss.detach().float()
    eval_preds.extend(
        tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
    )

  eval_epoch_loss = eval_loss/len(eval_dataloader)
  eval_ppl = torch.exp(eval_epoch_loss)
  train_epoch_loss = total_loss/len(train_dataloader)
  train_ppl = torch.exp(train_epoch_loss)
  print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl} {eval_epoch_loss}")


100%|██████████| 29/29 [00:07<00:00,  3.93it/s]


epoch=0: train_ppl=tensor(3.8584, device='cuda:0') train_epoch_loss=tensor(1.3503, device='cuda:0') 1.106655240058899 0.10134217888116837


100%|██████████| 29/29 [00:07<00:00,  4.02it/s]


epoch=1: train_ppl=tensor(1.1230, device='cuda:0') train_epoch_loss=tensor(0.1160, device='cuda:0') 1.0711097717285156 0.06869527697563171


100%|██████████| 29/29 [00:07<00:00,  4.05it/s]


epoch=2: train_ppl=tensor(1.0927, device='cuda:0') train_epoch_loss=tensor(0.0886, device='cuda:0') 1.044802188873291 0.04382754862308502


100%|██████████| 29/29 [00:07<00:00,  4.04it/s]


epoch=3: train_ppl=tensor(1.0762, device='cuda:0') train_epoch_loss=tensor(0.0735, device='cuda:0') 1.038212537765503 0.03750050440430641


100%|██████████| 29/29 [00:07<00:00,  4.03it/s]

epoch=4: train_ppl=tensor(1.0641, device='cuda:0') train_epoch_loss=tensor(0.0621, device='cuda:0') 1.0386053323745728 0.037878766655921936


In [ ]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset['validation']['text_label']):
  if pred.strip() == true.strip():
    correct += 1
  total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=96.47577092511013 % on the evaluation dataset
eval_preds[:10]=['positive', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral']
dataset['validation']['text_label'][:10]=['positive', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral']


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
peft_model_id = 'hang1n/t5-large_Prefix_Tuning_seq2seq'
model.push_to_hub("hang1n/t5-large_Prefix_Tuning_seq2seq", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hang1n/t5-large_Prefix_Tuning_seq2seq/commit/db29734fc5e67d7a5eb368daf913efc67e288402', commit_message='Upload model', commit_description='', oid='db29734fc5e67d7a5eb368daf913efc67e288402', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = 'hang1n/t5-large_Prefix_Tuning_seq2seq'

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(
    "The Lithuanian beer market made up 14.41 million liters in January , a rise of 0.8 percent from the year-earlier figure , the Lithuanian Brewers ' Association reporting citing the results from its members .",
    return_tensors="pt",
)

model.to(device)

with torch.no_grad():
  inputs = {k: v.to(device) for k,v in inputs.items()}
  outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=10)
  print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive']
